In [ ]:
import pandas as pd
import numpy as np

import json

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [ ]:
products = pd.read_pickle("./products_pd.pickle")

products['price_dollar'] = np.nan

In [ ]:
from fuzzyset import FuzzySet

def compute_match_score(product_name_list, store_name_list):
    
    if product_name_list and store_name_list:
        name_fs = FuzzySet(store_name_list)
        name_score = [0 if name_fs.get(i)==None else name_fs.get(i)[0][0] for i in product_name_list]
        return sum(name_score) / len(name_score)  
    else:
        return 0

# Walmart articles

In [ ]:
with open('walmart_products.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
walmart_products = pd.DataFrame.from_dict(data)

In [ ]:
walmart_products = walmart_products.drop_duplicates(subset='product_name')
walmart_products

In [ ]:
walmart_products['match_score'] = walmart_products.apply(lambda row: compute_match_score(row.product_name.lower().split(),
                                                       row.store_product_name.lower().replace('-', ' ').split()), 
                       axis=1)
walmart_products

# Migros articles

In [ ]:
with open('scraped_products.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
scraped_products = pd.DataFrame.from_dict(data)

In [ ]:
# Only two stores, all migros products are in CHF
migros_products = scraped_products[scraped_products['store_currency']=='CHF']

In [ ]:
from py_translator import Translator
translator = Translator()

# If there is no quantity, then it is no food item
migros_products = migros_products[migros_products['store_quantity']!='']


for row in migros_products.store_categories:
    translator = Translator()
    print(row[-1])
    translator.translate('Shortbread')


In [ ]:
from py_translator import Translator
s = Translator().translate(text='Hello my friend', dest='es').text
print(s)

# Monoprix articles

In [ ]:
migros_products